# Local LLM with Intel GPU v2

1. run Gemma model with ollama
2. create RESTful API Wrapper with flask
* create API key manager for API Wrapper 
* create wrapper for Oxygen Positron add-on by following:
    * https://www.oxygenxml.com/doc/ug-addons/topics/ai-positron-custom-connector.html
    * https://www.oxygenxml.com/doc/ug-addons/topics/ai_positron_enterprise.html
    * https://github.com/oxygenxml/oxygen-ai-positron-custom-connector-addon

I'll help you create a complete solution for running Gemma model with Ollama and creating a RESTful API wrapper with Flask, including an API key manager and Oxygen Positron connector wrapper.

## Step-by-step Solution:

1. **Set up Ollama with Gemma model** - Install dependencies and pull the Gemma model
2. **Create Flask API wrapper** - Build a RESTful API that interfaces with Ollama
3. **Implement API key manager** - Add authentication and API key validation
4. **Create Oxygen Positron connector** - Follow the specifications to create a compatible wrapper

### [localLlmWIntelGPUV2.ipynb](file:///c%3A/Project/OCR/localLlmWIntelGPUV2.ipynb)

Add implementation for running Gemma model with Ollama and creating Flask API wrapper with API key management and Oxygen Positron connector.



This implementation provides:

1. **Ollama Integration**: Automatic installation and Gemma model management
2. **Flask API Wrapper**: RESTful API with proper error handling
3. **API Key Management**: SQLite-based key storage with usage tracking
4. **Oxygen Positron Compatibility**: OpenAI-compatible endpoints that work with the Oxygen XML Author add-on
5. **Security**: API key authentication for all endpoints
6. **Monitoring**: Usage tracking and health checks

The solution follows the Oxygen Positron custom connector specifications and provides a complete local LLM setup with Intel GPU support via Ollama.

Similar code found with 1 license type

## Usage Instructions

### 1. Setup and Installation
- Run the notebook cell above to install dependencies and start the server
- The initial API key will be displayed - save it securely
- Ollama will be installed (if not present) and Gemma model will be pulled

### 2. API Endpoints

#### Standard Endpoints:
- `GET /health` - Health check
- `POST /api/generate` - Generate text with Ollama
- `GET /api/models` - List available models
- `POST /api/keys/generate` - Generate new API key

#### Oxygen Positron Compatible Endpoints:
- `POST /ai/chat/completions` - OpenAI-compatible chat completions
- `GET /ai/models` - OpenAI-compatible models list

### 3. Authentication
All endpoints (except health and key generation) require API key in header:
```
Authorization: Bearer your-api-key-here
```

### 4. Oxygen Positron Configuration
In Oxygen XML Author, configure the custom connector with:
- **Base URL**: `http://localhost:5000/ai`
- **API Key**: Your generated API key
- **Model**: `gemma3:4b`

### 5. Example Usage
```python
import requests

headers = {'Authorization': 'Bearer your-api-key'}
data = {
    "messages": [
        {"role": "user", "content": "Explain quantum computing"}
    ],
    "model": "gemma3:4b"
}

response = requests.post('http://localhost:5000/ai/chat/completions', 
                        json=data, headers=headers)
print(response.json())
```

In [1]:
# Install required dependencies
import subprocess
import sys
import os
import json
import uuid
import hashlib
import time
from datetime import datetime, timedelta
from functools import wraps
from flask import Flask, request, jsonify, g
import requests
import sqlite3

# Install required packages
def install_packages():
    packages = ['flask', 'requests', 'sqlite3']
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_packages()

# 1. Ollama Setup and Gemma Model Management
class OllamaManager:
    def __init__(self, base_url="http://localhost:11434"):
        self.base_url = base_url
        
    def install_ollama(self):
        """Install Ollama if not already installed"""
        try:
            subprocess.run(["ollama", "--version"], check=True, capture_output=True)
            print("Ollama is already installed")
        except (subprocess.CalledProcessError, FileNotFoundError):
            print("Installing Ollama...")
            # For Windows
            if os.name == 'nt':
                print("Please download and install Ollama from: https://ollama.ai/download")
            else:
                subprocess.run(["curl", "-fsSL", "https://ollama.ai/install.sh", "|", "sh"], shell=True)
    
    def pull_gemma_model(self, model_name="gemma3:4b"):
        """Pull Gemma model"""
        try:
            result = subprocess.run(["ollama", "pull", model_name], 
                                  capture_output=True, text=True, check=True)
            print(f"Successfully pulled {model_name}")
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error pulling model: {e}")
            return False
    
    def list_models(self):
        """List available models"""
        try:
            result = subprocess.run(["ollama", "list"], 
                                  capture_output=True, text=True, check=True)
            return result.stdout
        except subprocess.CalledProcessError as e:
            print(f"Error listing models: {e}")
            return None
    
    def generate_response(self, prompt, model="gemma3:4b", stream=False):
        """Generate response using Ollama API"""
        url = f"{self.base_url}/api/generate"
        payload = {
            "model": model,
            "prompt": prompt,
            "stream": stream
        }
        
        try:
            response = requests.post(url, json=payload)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error generating response: {e}")
            return None

# 2. API Key Manager
class APIKeyManager:
    def __init__(self, db_path="api_keys.db"):
        self.db_path = db_path
        self.init_database()
    
    def init_database(self):
        """Initialize SQLite database for API keys"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS api_keys (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                key_hash TEXT UNIQUE NOT NULL,
                name TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                last_used TIMESTAMP,
                is_active BOOLEAN DEFAULT 1,
                usage_count INTEGER DEFAULT 0,
                rate_limit INTEGER DEFAULT 100
            )
        ''')
        conn.commit()
        conn.close()
    
    def generate_api_key(self, name):
        """Generate a new API key"""
        api_key = f"sk-{uuid.uuid4().hex}"
        key_hash = hashlib.sha256(api_key.encode()).hexdigest()
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        try:
            cursor.execute('''
                INSERT INTO api_keys (key_hash, name) VALUES (?, ?)
            ''', (key_hash, name))
            conn.commit()
            return api_key
        except sqlite3.IntegrityError:
            return None
        finally:
            conn.close()
    
    def validate_api_key(self, api_key):
        """Validate API key and update usage"""
        key_hash = hashlib.sha256(api_key.encode()).hexdigest()
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            SELECT id, is_active, usage_count, rate_limit FROM api_keys 
            WHERE key_hash = ?
        ''', (key_hash,))
        
        result = cursor.fetchone()
        if result and result[1]:  # is_active
            # Update last_used and usage_count
            cursor.execute('''
                UPDATE api_keys 
                SET last_used = CURRENT_TIMESTAMP, usage_count = usage_count + 1
                WHERE key_hash = ?
            ''', (key_hash,))
            conn.commit()
            conn.close()
            return True
        
        conn.close()
        return False

# 3. Flask API Wrapper
app = Flask(__name__)
ollama_manager = OllamaManager()
api_key_manager = APIKeyManager()

def require_api_key(f):
    """Decorator to require API key authentication"""
    @wraps(f)
    def decorated_function(*args, **kwargs):
        api_key = request.headers.get('Authorization')
        if not api_key:
            return jsonify({'error': 'API key required'}), 401
        
        if api_key.startswith('Bearer '):
            api_key = api_key[7:]
        
        if not api_key_manager.validate_api_key(api_key):
            return jsonify({'error': 'Invalid API key'}), 401
        
        return f(*args, **kwargs)
    return decorated_function

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({'status': 'healthy', 'timestamp': datetime.utcnow().isoformat()})

@app.route('/api/generate', methods=['POST'])
@require_api_key
def api_generate():
    """Generate text using Ollama"""
    data = request.get_json()
    
    if not data or 'prompt' not in data:
        return jsonify({'error': 'Prompt is required'}), 400
    
    prompt = data['prompt']
    model = data.get('model', 'gemma3:4b')
    stream = data.get('stream', False)
    
    response = ollama_manager.generate_response(prompt, model, stream)
    
    if response:
        return jsonify(response)
    else:
        return jsonify({'error': 'Failed to generate response'}), 500

@app.route('/api/models', methods=['GET'])
@require_api_key
def api_list_models():
    """List available models"""
    models = ollama_manager.list_models()
    if models:
        return jsonify({'models': models})
    else:
        return jsonify({'error': 'Failed to list models'}), 500

@app.route('/api/keys/generate', methods=['POST'])
def generate_key():
    """Generate new API key (admin endpoint)"""
    data = request.get_json()
    if not data or 'name' not in data:
        return jsonify({'error': 'Name is required'}), 400
    
    api_key = api_key_manager.generate_api_key(data['name'])
    if api_key:
        return jsonify({'api_key': api_key})
    else:
        return jsonify({'error': 'Failed to generate API key'}), 500

# 4. Oxygen Positron Custom Connector
@app.route('/ai/chat/completions', methods=['POST'])
@require_api_key
def oxygen_chat_completions():
    """
    Oxygen Positron compatible endpoint
    Follows OpenAI Chat Completions API format
    """
    data = request.get_json()
    
    if not data or 'messages' not in data:
        return jsonify({'error': 'Messages are required'}), 400
    
    messages = data['messages']
    model = data.get('model', 'gemma3:4b')
    max_tokens = data.get('max_tokens', 150)
    temperature = data.get('temperature', 0.7)
    
    # Convert messages to a single prompt
    prompt = ""
    for message in messages:
        role = message.get('role', 'user')
        content = message.get('content', '')
        if role == 'system':
            prompt += f"System: {content}\n"
        elif role == 'user':
            prompt += f"User: {content}\n"
        elif role == 'assistant':
            prompt += f"Assistant: {content}\n"
    
    prompt += "Assistant: "
    
    # Generate response using Ollama
    response = ollama_manager.generate_response(prompt, model)
    
    if response and 'response' in response:
        # Format response in OpenAI Chat Completions format
        completion_response = {
            "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
            "object": "chat.completion",
            "created": int(time.time()),
            "model": model,
            "choices": [{
                "index": 0,
                "message": {
                    "role": "assistant",
                    "content": response['response']
                },
                "finish_reason": "stop"
            }],
            "usage": {
                "prompt_tokens": len(prompt.split()),
                "completion_tokens": len(response['response'].split()),
                "total_tokens": len(prompt.split()) + len(response['response'].split())
            }
        }
        return jsonify(completion_response)
    else:
        return jsonify({'error': 'Failed to generate response'}), 500

@app.route('/ai/models', methods=['GET'])
@require_api_key
def oxygen_list_models():
    """
    Oxygen Positron compatible models endpoint
    """
    models_output = ollama_manager.list_models()
    if models_output:
        # Parse and format models for Oxygen Positron
        model_list = {
            "object": "list",
            "data": [
                {
                    "id": "gemma3:4b",
                    "object": "model",
                    "created": int(time.time()),
                    "owned_by": "local"
                }
            ]
        }
        return jsonify(model_list)
    else:
        return jsonify({'error': 'Failed to list models'}), 500

# Initialize and run
if __name__ == '__main__':
    # Setup Ollama and Gemma
    print("Setting up Ollama and Gemma model...")
    ollama_manager.install_ollama()
    # ollama_manager.pull_gemma_model()
    ollama_manager.list_models()
    
    # Generate initial API key
    initial_key = api_key_manager.generate_api_key("default")
    print(f"Initial API Key: {initial_key}")
    print("Save this key - it won't be shown again!")
    
    # Start Flask app
    print("Starting Flask API server...")
    app.run(host='0.0.0.0', port=5000, debug=True)

Setting up Ollama and Gemma model...
Ollama is already installed
Initial API Key: sk-1e2baeca84d14e9e9b1ee980cc1db424
Save this key - it won't be shown again!
Starting Flask API server...
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.6:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Project\OCR\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%tb

SystemExit: 1

You can use the selected health check endpoint in several ways to monitor your Flask API:

## 1. Test Health Check via HTTP Request



In [ ]:
import requests

# Test the health endpoint
try:
    response = requests.get('http://localhost:5000/health')
    if response.status_code == 200:
        health_data = response.json()
        print(f"Service Status: {health_data['status']}")
        print(f"Timestamp: {health_data['timestamp']}")
    else:
        print(f"Health check failed with status: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"Service is unreachable: {e}")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /health HTTP/1.1" 200 143


Service Status: healthy
Timestamp: 2025-06-01T06:25:27.852527




## 2. Browser-based Health Check

Simply open your browser and navigate to:


In [ ]:
http://localhost:5000/health



You should see a JSON response like:


In [ ]:
{
  "status": "healthy",
  "timestamp": "2025-05-31T14:30:45.123456"
}



## 3. Enhance the Health Check Endpoint

You can improve the health check to include more diagnostic information:



In [ ]:
@app.route('/health', methods=['GET'])
def health_check():
    """Enhanced health check endpoint"""
    try:
        # Check Ollama connectivity
        ollama_status = "unknown"
        try:
            models = ollama_manager.list_models()
            ollama_status = "healthy" if models else "unhealthy"
        except:
            ollama_status = "unhealthy"
        
        # Check database connectivity
        db_status = "unknown"
        try:
            conn = sqlite3.connect(api_key_manager.db_path)
            conn.close()
            db_status = "healthy"
        except:
            db_status = "unhealthy"
        
        overall_status = "healthy" if ollama_status == "healthy" and db_status == "healthy" else "degraded"
        
        return jsonify({
            'status': overall_status,
            'timestamp': datetime.utcnow().isoformat(),
            'services': {
                'ollama': ollama_status,
                'database': db_status
            }
        })
    except Exception as e:
        return jsonify({
            'status': 'unhealthy',
            'timestamp': datetime.utcnow().isoformat(),
            'error': str(e)
        }), 500



## 4. Automated Health Monitoring Script

Create a monitoring script that regularly checks the health:



In [ ]:
import requests
import time
import logging

def monitor_health(url="http://localhost:5000/health", interval=30):
    """Monitor health endpoint every 30 seconds"""
    logging.basicConfig(level=logging.INFO)
    
    while True:
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                data = response.json()
                logging.info(f"✅ Service healthy - {data['timestamp']}")
            else:
                logging.warning(f"❌ Health check returned {response.status_code}")
        except Exception as e:
            logging.error(f"🔥 Health check failed: {e}")
        
        time.sleep(interval)

# Run the monitor
monitor_health()



## 5. Command Line Health Check

Use curl or PowerShell to check health from command line:

**Using curl:**


In [ ]:
curl http://localhost:5000/health



**Using PowerShell:**


In [ ]:
Invoke-RestMethod -Uri "http://localhost:5000/health" -Method GET



## 6. Integration with Load Balancers

The health endpoint is designed to work with load balancers and container orchestrators. Configure your load balancer to:
- **Health Check URL**: `http://localhost:5000/health`
- **Expected Status Code**: `200`
- **Check Interval**: `30 seconds`
- **Timeout**: `5 seconds`

This health check endpoint helps ensure your local LLM API service is running properly and can respond to requests from the Oxygen Positron connector.

## Plan

The code has a SystemExit error which is likely caused by running Flask in a Jupyter notebook. Flask's `app.run()` is designed for standalone scripts, not interactive environments. I need to:

1. Modify the initialization section to work in Jupyter
2. Add proper error handling for Ollama installation
3. Make the Flask app run in a way that's compatible with Jupyter notebooks
4. Add better model list parsing for the oxygen_list_models endpoint
5. Ensure proper cleanup and threading



In [ ]:
# Install required dependencies
import subprocess
import sys
import os
import json
import uuid
import hashlib
import time
import threading
from datetime import datetime, timedelta
from functools import wraps
from flask import Flask, request, jsonify, g
import requests
import sqlite3
import logging

ollama_default_model = "gemma3:1b"

# Install required packages
def install_packages():
    packages = ['flask', 'requests']
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_packages()

# 1. Ollama Setup and Gemma Model Management
class OllamaManager:
    def __init__(self, base_url="http://localhost:11434"):
        self.base_url = base_url
        
    def install_ollama(self):
        """Install Ollama if not already installed"""
        try:
            result = subprocess.run(["ollama", "--version"], check=True, capture_output=True, text=True)
            print("Ollama is already installed")
            return True
        except (subprocess.CalledProcessError, FileNotFoundError):
            print("Installing Ollama...")
            # For Windows
            if os.name == 'nt':
                print("Please download and install Ollama from: https://ollama.ai/download")
                print("After installation, restart this notebook and run 'ollama serve' in a terminal")
                return False
            else:
                try:
                    # Download installer script
                    import urllib.request
                    urllib.request.urlretrieve("https://ollama.ai/install.sh", "ollama_install.sh")
                    subprocess.run(["chmod", "+x", "ollama_install.sh"], check=True)
                    subprocess.run(["./ollama_install.sh"], check=True)
                    print("Ollama installed successfully")
                    return True
                except Exception as e:
                    print(f"Failed to install Ollama: {e}")
                    return False
    
    def pull_gemma_model(self, model_name=ollama_default_model):
        """Pull Gemma model"""
        try:
            print(f"Pulling {model_name} model...")
            result = subprocess.run(["ollama", "pull", model_name], 
                                  capture_output=True, text=True, check=True)
            print(f"Successfully pulled {model_name}")
            return True
        except subprocess.CalledProcessError as e:
            print(f"Error pulling model: {e}")
            print(f"Make sure Ollama is running with 'ollama serve'")
            return False
    
    def list_models(self):
        """List available models"""
        try:
            result = subprocess.run(["ollama", "list"], 
                                  capture_output=True, text=True, check=True)
            return result.stdout
        except subprocess.CalledProcessError as e:
            print(f"Error listing models: {e}")
            return None
    
    def parse_models(self, models_output):
        """Parse ollama list output into structured format"""
        if not models_output:
            return []
        
        models = []
        lines = models_output.strip().split('\n')[1:]  # Skip header
        for line in lines:
            if line.strip():
                parts = line.split()
                if len(parts) >= 3:
                    model_name = parts[0]
                    models.append({
                        "id": model_name,
                        "object": "model",
                        "created": int(time.time()),
                        "owned_by": "local"
                    })
        return models
    
    def generate_response(self, prompt, model=ollama_default_model, stream=False):
        """Generate response using Ollama API with extended timeout"""
        url = f"{self.base_url}/api/generate"
        payload = {
            "model": model,
            "prompt": prompt,
            "stream": stream,
            "options": {
                "num_predict": 3000,  # Limit tokens to speed up
                "temperature": 0.7
            }
        }
        
        # Use longer timeout for qwen models
        if "qwen" in model.lower():
            timeout = 480 
        elif "deepseek" in model.lower():
            timeout = 180 
        else:
            timeout = 60
        # timeout = 480 if "qwen" in model.lower() or "deepseek" in model.lower() else 60
        
        try:
            print(f"Generating with {model} (timeout: {timeout}s)...")
            response = requests.post(url, json=payload, timeout=timeout)
            response.raise_for_status()
            # print(json.dumps(response.json(), indent=2))
            result = response.json()
            if 'response' in result:
                print(result['response'])
            else:
                print(json.dumps(result, indent=2))
            return response.json()
        except requests.exceptions.Timeout:
            print(f"Timeout after {timeout} seconds with model {model}")
            return None
        except requests.exceptions.RequestException as e:
            print(f"Error generating response: {e}")
            return None

# 2. API Key Manager
class APIKeyManager:
    def __init__(self, db_path="api_keys.db"):
        self.db_path = db_path
        self.init_database()
    
    def init_database(self):
        """Initialize SQLite database for API keys"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS api_keys (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                key_hash TEXT UNIQUE NOT NULL,
                name TEXT NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                last_used TIMESTAMP,
                is_active BOOLEAN DEFAULT 1,
                usage_count INTEGER DEFAULT 0,
                rate_limit INTEGER DEFAULT 100
            )
        ''')
        conn.commit()
        conn.close()
    
    def generate_api_key(self, name):
        """Generate a new API key"""
        api_key = f"sk-{uuid.uuid4().hex}"
        key_hash = hashlib.sha256(api_key.encode()).hexdigest()
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        try:
            cursor.execute('''
                INSERT INTO api_keys (key_hash, name) VALUES (?, ?)
            ''', (key_hash, name))
            conn.commit()
            return api_key
        except sqlite3.IntegrityError:
            return None
        finally:
            conn.close()
    
    def validate_api_key(self, api_key):
        """Validate API key and update usage"""
        key_hash = hashlib.sha256(api_key.encode()).hexdigest()
        
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''
            SELECT id, is_active, usage_count, rate_limit FROM api_keys 
            WHERE key_hash = ?
        ''', (key_hash,))
        
        result = cursor.fetchone()
        if result and result[1]:  # is_active
            # Update last_used and usage_count
            cursor.execute('''
                UPDATE api_keys 
                SET last_used = CURRENT_TIMESTAMP, usage_count = usage_count + 1
                WHERE key_hash = ?
            ''', (key_hash,))
            conn.commit()
            conn.close()
            return True
        
        conn.close()
        return False

# 3. Flask API Wrapper
app = Flask(__name__)
ollama_manager = OllamaManager()
api_key_manager = APIKeyManager()

def require_api_key(f):
    """Decorator to require API key authentication"""
    @wraps(f)
    def decorated_function(*args, **kwargs):
        api_key = request.headers.get('Authorization')
        if not api_key:
            return jsonify({'error': 'API key required'}), 401
        
        if api_key.startswith('Bearer '):
            api_key = api_key[7:]
        
        if not api_key_manager.validate_api_key(api_key):
            return jsonify({'error': 'Invalid API key'}), 401
        
        return f(*args, **kwargs)
    return decorated_function

@app.route('/health', methods=['GET'])
def health_check():
    """Enhanced health check endpoint"""
    try:
        # Check Ollama connectivity
        ollama_status = "unknown"
        try:
            models = ollama_manager.list_models()
            ollama_status = "healthy" if models else "unhealthy"
        except:
            ollama_status = "unhealthy"
        
        # Check database connectivity
        db_status = "unknown"
        try:
            conn = sqlite3.connect(api_key_manager.db_path)
            conn.close()
            db_status = "healthy"
        except:
            db_status = "unhealthy"
        
        overall_status = "healthy" if ollama_status == "healthy" and db_status == "healthy" else "degraded"
        
        return jsonify({
            'status': overall_status,
            'timestamp': datetime.utcnow().isoformat(),
            'services': {
                'ollama': ollama_status,
                'database': db_status
            }
        })
    except Exception as e:
        return jsonify({
            'status': 'unhealthy',
            'timestamp': datetime.utcnow().isoformat(),
            'error': str(e)
        }), 500

@app.route('/api/generate', methods=['POST'])
@require_api_key
def api_generate():
    """Generate text using Ollama"""
    data = request.get_json()
    
    if not data or 'prompt' not in data:
        return jsonify({'error': 'Prompt is required'}), 400
    
    prompt = data['prompt']
    model = data.get('model', 'gemma3:4b')
    stream = data.get('stream', False)
    
    response = ollama_manager.generate_response(prompt, model, stream)
    
    if response:
        return jsonify(response)
    else:
        return jsonify({'error': 'Failed to generate response'}), 500

@app.route('/api/models', methods=['GET'])
@require_api_key
def api_list_models():
    """List available models"""
    models_output = ollama_manager.list_models()
    if models_output:
        parsed_models = ollama_manager.parse_models(models_output)
        return jsonify({'models': parsed_models})
    else:
        return jsonify({'error': 'Failed to list models'}), 500

@app.route('/api/keys/generate', methods=['POST'])
def generate_key():
    """Generate new API key (admin endpoint)"""
    data = request.get_json()
    if not data or 'name' not in data:
        return jsonify({'error': 'Name is required'}), 400
    
    api_key = api_key_manager.generate_api_key(data['name'])
    if api_key:
        return jsonify({'api_key': api_key})
    else:
        return jsonify({'error': 'Failed to generate API key'}), 500

# 4. Oxygen Positron Custom Connector
@app.route('/ai/chat/completions', methods=['POST'])
@require_api_key
def oxygen_chat_completions():
    """
    Oxygen Positron compatible endpoint
    Follows OpenAI Chat Completions API format`
    """
    data = request.get_json()
    print(f"Received data: {data}")
    
    if not data or 'messages' not in data:
        return jsonify({'error': 'Messages are required'}), 400
    
    messages = data['messages']
    model = data.get('model', ollama_default_model)
    max_tokens = data.get('max_tokens', 150)
    temperature = data.get('temperature', 0.7)
    
    # Convert messages to a single prompt
    prompt = ""
    for message in messages:
        role = message.get('role', 'user')
        content = message.get('content', '')
        if role == 'system':
            prompt += f"System: {content}\n"
        elif role == 'user':
            prompt += f"User: {content}\n"
        elif role == 'assistant':
            prompt += f"Assistant: {content}\n"
    
    prompt += "Assistant: "
    
    # Generate response using Ollama
    response = ollama_manager.generate_response(prompt, model)
    
    if response and 'response' in response:
        # Format response in OpenAI Chat Completions format
        completion_response = {
            "id": f"chatcmpl-{uuid.uuid4().hex[:8]}",
            "object": "chat.completion",
            "created": int(time.time()),
            "model": model,
            "choices": [{
                "index": 0,
                "message": {
                    "role": "assistant",
                    "content": response['response']
                },
                "finish_reason": "stop"
            }],
            "usage": {
                "prompt_tokens": len(prompt.split()),
                "completion_tokens": len(response['response'].split()),
                "total_tokens": len(prompt.split()) + len(response['response'].split())
            }
        }
        return jsonify(completion_response)
    else:
        return jsonify({'error': 'Failed to generate response'}), 500

@app.route('/ai/models', methods=['GET'])
@require_api_key
def oxygen_list_models():
    """
    Oxygen Positron compatible models endpoint
    """
    models_output = ollama_manager.list_models()
    if models_output:
        # Parse and format models for Oxygen Positron
        parsed_models = ollama_manager.parse_models(models_output)
        if not parsed_models:
            # Fallback if no models found
            parsed_models = [{
                "id": ollama_default_model,
                "object": "model",
                "created": int(time.time()),
                "owned_by": "local"
            }]
        
        model_list = {
            "object": "list",
            "data": parsed_models
        }
        return jsonify(model_list)
    else:
        return jsonify({'error': 'Failed to list models'}), 500

# Global variable to track server thread
server_thread = None

def run_flask_app():
    """Run Flask app in a separate thread"""
    # Enable Flask debug logging
    logging.basicConfig(level=logging.DEBUG)
    app.logger.setLevel(logging.DEBUG)
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

def start_server():
    """Start the Flask server"""
    global server_thread
    if server_thread is None or not server_thread.is_alive():
        server_thread = threading.Thread(target=run_flask_app, daemon=True)
        server_thread.start()
        print("Flask server started on http://localhost:5000")
    else:
        print("Server is already running")

def setup_and_start():
    """Setup Ollama and start the Flask server"""
    print("Setting up Ollama and Gemma model...")
    
    # Install Ollama
    if not ollama_manager.install_ollama():
        print("⚠️  Please install Ollama manually and run 'ollama serve' before continuing")
        return None
    
    # Try to list models (this will also test if Ollama is running)
    models = ollama_manager.list_models()
    if models:
        print("Available models:")
        print(models)
    else:
        print("⚠️  No models found. You may need to pull a model first.")
        print(f"Run: ollama pull {ollama_default_model}")
    
    # Generate initial API key
    # initial_key = api_key_manager.generate_api_key("default")
    # if initial_key:
    #     print(f"\n🔑 Initial API Key: {initial_key}")
    #     print("Save this key - it won't be shown again!")
    
    # Start Flask app in background thread
    start_server()
    
    # return initial_key

# Initialize setup
if __name__ == '__main__':
    setup_and_start()
else:
    # When imported in Jupyter, just run setup
    initial_key = setup_and_start()
    if initial_key:
        print(f"\n✅ Server setup complete!")
        print(f"🌐 Health check: http://localhost:5000/health")
        print(f"📚 API documentation available in the notebook cells above")

Setting up Ollama and Gemma model...
Ollama is already installed
Available models:
NAME                ID              SIZE      MODIFIED       
deepseek-r1:1.5b    e0979632db5a    1.1 GB    22 minutes ago    
gemma3:1b           8648f39daa8f    815 MB    46 hours ago      
gemma3:4b           a2af6cc3eb7f    3.3 GB    2 days ago        

Flask server started on http://localhost:5000
 * Serving Flask app '__main__'


 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.6:5000
INFO:werkzeug:Press CTRL+C to quit
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434


Received data: {'model': 'deepseek-r1:1.5b', 'messages': [{'role': 'system', 'content': "${contextInfo} You're a XML markup editor. You should markup the selected string which is bibliography from Buddist Tripitaka  with TEI P5 tags by the following: 1) enclosed the string with <ref> for bibliorgraphy reference; 2) follow the examples to markup the rest semantical elements of <canon> for canon, <v>for volume, <w>for work(after 'No.' normally), <p> for page(after 'p' or '頁'), <c> for column(in a, b, c or 上段, 中段, 下段) and <l> for line. 3) You are only allowed to add the above xml tags without adding extra characters or punctuations, also leave the rest untagged characters between tags. 5) if the first char is 't' replace it with 'T' or if the first three char is 'cht' or 'chT' replace it with 'ChT'."}, {'role': 'user', 'content': 'p.69c7'}, {'role': 'assistant', 'content': '<ref>p.<p>69</p><c>c</c><l>7</l></ref>'}, {'role': 'user', 'content': '『大正藏』第四卷359a1-b7'}, {'role': 'assistant', 'co

DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/1.1" 200 None
INFO:werkzeug:127.0.0.1 - - [02/Jun/2025 00:21:46] "POST /ai/chat/completions HTTP/1.1" 200 -


...

<ref>p.<p>67</p><c>c</c><l>4</l></ref>


DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434


Received data: {'model': 'gemma3:4b', 'messages': [{'role': 'system', 'content': "${contextInfo} You're a XML markup editor. You should markup the selected string which is bibliography from Buddist Tripitaka  with TEI P5 tags by the following: 1) enclosed the string with <ref> for bibliorgraphy reference; 2) follow the examples to markup the rest semantical elements of <canon> for canon, <v>for volume, <w>for work(after 'No.' normally), <p> for page(after 'p' or '頁'), <c> for column(in a, b, c or 上段, 中段, 下段) and <l> for line. 3) You are only allowed to add the above xml tags without adding extra characters or punctuations, also leave the rest untagged characters between tags. 5) if the first char is 't' replace it with 'T' or if the first three char is 'cht' or 'chT' replace it with 'ChT'."}, {'role': 'user', 'content': 'p.69c7'}, {'role': 'assistant', 'content': '<ref>p.<p>69</p><c>c</c><l>7</l></ref>'}, {'role': 'user', 'content': '『大正藏』第四卷359a1-b7'}, {'role': 'assistant', 'content':

DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/1.1" 200 None
INFO:werkzeug:127.0.0.1 - - [02/Jun/2025 00:38:09] "POST /ai/chat/completions HTTP/1.1" 200 -


```xml
<ref><canon>T</canon>.<v>22</v> No.<w>366</w> p.<p>348</p><c>b</c><l>9</l></ref>
```


In [ ]:
import subprocess

def remove_ollama_model(model_name):
    """Remove a specific model from Ollama"""
    try:
        print(f"Removing model: {model_name}")
        result = subprocess.run(['ollama', 'rm', model_name], 
                              capture_output=True, text=True, shell=True)
        
        if result.returncode == 0:
            print(f"✅ Model {model_name} removed successfully!")
            print(result.stdout)
            return True
        else:
            print(f"❌ Failed to remove model {model_name}")
            print(f"Error: {result.stderr}")
            return False
            
    except Exception as e:
        print(f"❌ Error removing model: {e}")
        return False

# Example: Remove gemma3:4b model
remove_ollama_model('qwen3:4b')

# # Example: Remove gemma3:1b model
# remove_ollama_model('gemma3:1b')

Removing model: qwen3:4b
✅ Model qwen3:4b removed successfully!
deleted 'qwen3:4b'



True

  * p.67c9
  
  ```
  <think>
Okay, let's tackle this user query. The user provided the string "p.67c4" and wants it formatted according to the given rules. Let me go through the steps carefully.

First, check the first character. The string starts with 'p', which is not a 't', so step 1 doesn't apply here. But wait, the user mentioned that if the first three characters are 'cht' or 'chT', replace with 'ChT'. Here, the first character is 'p', so that's not relevant. So no changes needed for the first part.

Next, enclose the entire string in <ref> tags. So the outermost tag is <ref>...</ref>.

Now, the rest of the elements. The string is "p.67c4". Breaking it down:

- "p." is the page indicator. According to the examples, after 'p' comes <p>67</p>. So that's straightforward.

- Then there's "c4". The 'c' stands for column, so that's <c>c</c>, and the '4' is the line number, so <l>4</l>.

Putting it all together, the structure should be:

<ref>p.<p>67</p><c>c</c><l>4</l></ref>

Wait, but in the examples, sometimes the 'c' is followed by a space, like in the example where it was "a b7". But in the user's previous examples, like "p.69c7", the assistant used <c>c</c><l>7</l>. So maybe the 'c' is directly followed by the line number without a space. Let me check the examples again.

Looking at the user's example: "p.69c7" becomes <ref>p.<p>69</p><c>c</c><l>7</l></ref>. So yes, no space between c and the line number. Therefore, in this case, "c4" should be <c>c</c><l>4</l>.

So the final answer should be as above. Let me make sure there are no other elements. The string is "p.67c4", so after 'p.' comes 67, then column c, then line 4. No other parts. So the XML should be correctly structured with the tags in order.
</think>

<ref>p.<p>67</p><c>c</c><l>4</l></ref>
  ```

* t.No.366,p.348b9

```
<think>
Okay, let's tackle this user query. The user provided the string "t.No.366,p.348b9" and wants it formatted according to the specified rules.

First, I need to check the first character. It starts with 't', so according to rule 1, I should replace the first character with 'T'. So that becomes "T.No.366,p.348b9".

Next, the entire string needs to be enclosed in <ref> tags. So the outer structure will be <ref>...</ref>.

Now, breaking down the components. The "T" is the canon, so that's <canon>T</canon>. Then "No.366" – the "No." is part of the volume, but wait, the example shows "No." as a <w> tag. Wait, looking at previous examples, like in the user's example where "No." was followed by a work number. For instance, in the third example: "<canon>大正</canon><v>18</v> No.<w>865</w>..." So "No." is followed by a <w> tag. But in the current case, "No.366" – the "No." is part of the volume, but the "366" is the work number. Wait, maybe "No." is the work number, so "No.366" would be <w>366</w>? Wait, but in the example, "No." is followed by <w>865</w>. So "No." is the work number, so "No.366" would be <w>366</w>? Or is "No." a separate tag?

Wait, looking at the user's example: "大正18 no.865 p.216.b.4-12" becomes "<canon>大正</canon><v>18</v> No.<w>865</w> p.<p>216</p>.<c>b</c>.<l>4</l>-<l>12</l>". So "No." is followed by <w>865</w>. Therefore, in the current case, "No.366" would be "No." followed by <w>366</w>? Wait, but the original is "t.No.366,p.348b9". So the "No.366" is part of the work number. So "No." is the work number, and "366" is the specific work. Therefore, it should be "No.<w>366</w>".

Then, the rest is "p.348b9". The "p." is page, so <p>348</p>, then "b9" would be <c>b</c><l>9</l>.

Putting it all together: <ref><canon>T</canon> No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>.

Wait, but in the previous example, "t.No.366,p.348b9" was processed as "<ref><canon>T</canon>.<v>22</v>,<p>761</p><c>a</c><l>21</l>-<l>29</l></ref>". Wait, but in that case, the "No." was not tagged. Wait, maybe there's inconsistency here. Let me check the previous examples again.

Looking at the user's example: "t.No.366,p.348b9" – the assistant's answer was <ref><canon>T</canon>.<v>22</v>,<p>761</p><c>a</c><l>21</l>-<l>29</l></ref>. Wait, but that example was for "t.22,761a21-29", which was converted to <ref><canon>T</canon>.<v>22</v>,<p>761</p><c>a</c><l>21</l>-<l>29</l></ref>. So in that case, the "No." was not present. So perhaps the "No." is part of the volume? Wait, but in the user's example, "大正18 no.865 p.216.b.4-12" becomes "<canon>大正</canon><v>18</v> No.<w>865</w> p.<p>216</p>.<c>b</c>.<l>4</l>-<l>12</l>". So in that case, "No." is followed by <w>865</w>, which is the work number.

So in the current case, "t.No.366,p.348b9", the "No." is part of the work number. So "No.366" would be "No." followed by <w>366</w>. So the correct structure would be:

<ref><canon>T</canon> No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>

But wait, in the previous example where "t.22,761a21-29" became "<ref><canon>T</canon>.<v>22</v>,<p>761</p><c>a</c><l>21</l>-<l>29</l></ref>", there's no "No." part. So maybe "No." is only present when it's a work number. Therefore, in the current case, "t.No.366,p.348b9" is "T" as the canon, then "No.366" as the work number, then "p.348b9".

But the problem is that in the previous example where "t.No.366,p.348b9" was processed, but maybe that's a different case. Wait, the user's example for "t.No.366,p.348b9" is the current query. Wait, the user's current input is "t.No.366,p.348b9", and the assistant is to process it.

So, the first character is 't', so replace with 'T'. Then, the "No.366" is part of the work number. So "No." is followed by <w>366</w>. Then, "p.348b9" is <p>348</p><c>b</c><l>9</l>.

Putting it all together, the XML should be:

<ref><canon>T</canon> No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>

But in the previous example where "t.22,761a21-29" was converted, there was no "No." part, so perhaps "No." is only present when it's a work number. So in this case, "No.366" would be "No." followed by <w>366</w>, and then the rest is p.348b9.

So that's the correct structure. Therefore, the answer should be as above.
</think>

<ref><canon>T</canon> No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>
```

In [ ]:
import requests
import json

# Replace with your actual API key
api_key = "sk-2c76be0b113d4e54aa129a3289eea8b5"
headers = {'Authorization': f'Bearer {api_key}'}

response = requests.get('http://localhost:5000/api/models', headers=headers)
print(json.dumps(response.json(), indent=2))

response = requests.get('http://localhost:5000/ai/models', headers=headers)
print(json.dumps(response.json(), indent=2))



DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000
DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /api/models HTTP/1.1" 200 374
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:5000


{
  "models": [
    {
      "created": 1748761670,
      "id": "qwen3:4b",
      "object": "model",
      "owned_by": "local"
    },
    {
      "created": 1748761670,
      "id": "gemma3:1b",
      "object": "model",
      "owned_by": "local"
    },
    {
      "created": 1748761670,
      "id": "gemma3:4b",
      "object": "model",
      "owned_by": "local"
    }
  ]
}


DEBUG:urllib3.connectionpool:http://localhost:5000 "GET /ai/models HTTP/1.1" 200 392


{
  "data": [
    {
      "created": 1748761672,
      "id": "qwen3:4b",
      "object": "model",
      "owned_by": "local"
    },
    {
      "created": 1748761672,
      "id": "gemma3:1b",
      "object": "model",
      "owned_by": "local"
    },
    {
      "created": 1748761672,
      "id": "gemma3:4b",
      "object": "model",
      "owned_by": "local"
    }
  ],
  "object": "list"
}


## setting


1. Plugins / Oxygen AI Positron Assistant Enterprise / AI Service Configuration
* AI connector: `Custom AI service`
* Address: `http://127.0.0.1:5000/ai/`
* API key: `from flask API key manager`
* Model: `by selection from gemma3:1b or gemma3:4b`
* Enable text moderation: `unchecked`
* Enable streaming: `checked`


## result

| AI Positron | model | Action\xml | p.67c4 | t.No.366,p.348b9 |
|----------|----------|----------|----------|----------|
| AI Positron |GPT-4o|ref markup++| ✅```<ref>p.<p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>T</canon>.No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>``` |
|  |GPT-4o mini | | ✅```<ref>p.<p>67</p><c>c</c><l>4</l></ref>``` | ✅```<ref><canon>T</canon>.No.<w>366</w>,p.<p>348</p><c>b</c><l>9</l></ref>``` |
| AI Positron<br>Enterprise |gemma3:4b| | ✅```<ref>p.<p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>T</canon>.<v>22</v> No.<w>366</w> p.<p>348</p><c>b</c><l>9</l></ref>``` |
|  |gemma3:1b| | ✅```<ref>p.<p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>No.<w>366</w> p.<p>348</p>.<c>b</c>.<l>9</l></ref>``` |
|  |qwen3:4b(T)| | ✅```<ref>p.<p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>T</canon> No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>``` |
|  |deepseek-r1:1.5b(T)| | ✅```<ref>p.<p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>t.No.366,p.348b9</canon></ref><v>366</v><p>.348</p><c>b</c><l>9</l>``` |
| AI Positron |GPT-4o|ref markup+| ❌```<ref><p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>T</canon>.No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>``` |
|  |GPT-4o mini| | ❌```<ref><p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>T</canon>.No.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>``` |
| AI Positron<br>Enterprise |gemma3:4b<br>| | ❌```<ref><canon>ChT</canon>.<p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>T</canon>.<v>366</v>,<p>348</p><c>b</c><l>9</l></ref>``` |
|  |gemma3:1b| | ❌```<ref><canon>p</canon>.<v>67</v>,<p>4</p><c>4</c>.<l>4</l></ref>``` | ❌```<ref><canon>No.</canon>366.<v>348</v>,<p>b</p><c>9</c>.<l>34</l></ref>``` |
|  |qwen3:4b(T)| | ❌```<ref><p>67</p><c>c</c><l>4</l></ref>``` | ❌```<ref><canon>T</canon>.<w>366</w>,<p>348</p><c>b</c><l>9</l></ref>``` |
|  |deepseek-r1:1.5b(T)| | ❌```<ref><canon>p.67</canon><v>67</v><p>67</p><c>4</c><l>1</l>-<l>296</l></ref>``` | ❌```<ref><canon>T</canon>.<v>366</v>,<p>348</p><c>b</c><l>0</l>-<l>1</l></ref>``` |



In [ ]:
ollama_manager.pull_gemma_model("deepseek-r1:1.5b")


Pulling deepseek-r1:1.5b model...


Exception in thread Thread-25 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\jeffw\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Project\OCR\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\jeffw\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jeffw\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp950' codec can't decode byte 0xe2 in position 35: illegal multibyte sequence


Successfully pulled deepseek-r1:1.5b


True

In [ ]:
ollama_manager.pull_gemma_model("qwen3:4b")


Pulling qwen3:4b model...


Exception in thread Thread-34 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\jeffw\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Project\OCR\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\jeffw\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\jeffw\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
UnicodeDecodeError: 'cp950' codec can't decode byte 0xe2 in position 35: illegal multibyte sequence


Successfully pulled qwen3:4b


True

In [ ]:
model_list = ollama_manager.list_models()
print(json.dumps(model_list, indent=2))

# response = ollama_manager.generate_response("Hello, how are you?", model="qwen3:4b")
# # print(json.dumps(response, indent=2))
# print(response.json()['response'])

# response = ollama_manager.generate_response("What is the capital of France?", model="qwen3:4b")
# print(response.json()['response'])

response = ollama_manager.generate_response("Hello, how are you?", model="gemma3:4b")
print(response.json()['response'])

response = ollama_manager.generate_response("What is the capital of France?", model="gemma3:4b")
print(response.json()['response'])

response = ollama_manager.generate_response("Hello, how are you?", model="gemma3:1b")
print(response.json()['response'])

response = ollama_manager.generate_response("What is the capital of France?", model="gemma3:1b")
print(response.json()['response'])

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434


"NAME         ID              SIZE      MODIFIED     \ngemma3:1b    8648f39daa8f    815 MB    46 hours ago    \ngemma3:4b    a2af6cc3eb7f    3.3 GB    2 days ago      \n"
Generating with gemma3:4b (timeout: 60s)...


DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/1.1" 200 1027


I’m doing well, thanks for asking! As a large language model, I don’t really *feel* in the same way humans do, but my systems are running smoothly and I’m ready to chat. 😊 

How are *you* doing today? Is there anything you’d like to talk about or anything I can help you with?


AttributeError: 'dict' object has no attribute 'json'

Collecting workspace informationBased on the error message and your notebook output, the issue with `qwen3:4b` getting HTTP 500 errors while `gemma3:4b` works fine is likely due to several factors:

## 1. **Model Size and Resource Requirements**

The `qwen3:4b` model may have different resource requirements than `gemma3:4b`:



In [ ]:
# Check model sizes and resource usage
def check_model_info():
    """Check detailed model information"""
    try:
        # Get detailed model info
        result = subprocess.run(["ollama", "show", "qwen3:4b"], 
                              capture_output=True, text=True, check=True)
        print("Qwen3:4b model info:")
        print(result.stdout)
        
        result = subprocess.run(["ollama", "show", "gemma3:4b"], 
                              capture_output=True, text=True, check=True)
        print("\nGemma3:4b model info:")
        print(result.stdout)
        
    except subprocess.CalledProcessError as e:
        print(f"Error getting model info: {e}")

check_model_info()

Qwen3:4b model info:
  Model
    architecture        qwen3     
    parameters          4.0B      
    context length      40960     
    embedding length    2560      
    quantization        Q4_K_M    

  Capabilities
    completion    
    tools         
    thinking      

  Parameters
    stop              "<|im_start|>"    
    stop              "<|im_end|>"      
    temperature       0.6               
    top_k             20                
    top_p             0.95              
    repeat_penalty    1                 

  License
    Apache License               
    Version 2.0, January 2004    
    ...                          



Gemma3:4b model info:
  Model
    architecture        gemma3    
    parameters          4.3B      
    context length      131072    
    embedding length    2560      
    quantization        Q4_K_M    

  Capabilities
    completion    
    vision        

  Parameters
    top_k          64                 
    top_p          0.95            



## 2. **Timeout Issues**

The error shows a read timeout after 60 seconds. `qwen3:4b` might be slower to respond:



In [ ]:
def generate_response_with_longer_timeout(self, prompt, model=ollama_default_model, stream=False):
    """Generate response using Ollama API with extended timeout"""
    url = f"{self.base_url}/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": stream,
        "options": {
            "num_predict": 100,  # Limit tokens to speed up
            "temperature": 0.7
        }
    }
    
    # Use longer timeout for qwen models
    timeout = 180 if "qwen" in model.lower() else 60
    
    try:
        print(f"Generating with {model} (timeout: {timeout}s)...")
        response = requests.post(url, json=payload, timeout=timeout)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.Timeout:
        print(f"Timeout after {timeout} seconds with model {model}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"Error generating response: {e}")
        return None

# Update your OllamaManager class
ollama_manager.generate_response = generate_response_with_longer_timeout.__get__(ollama_manager, OllamaManager)



## 3. **Model Loading State**

`qwen3:4b` might not be fully loaded into memory:



In [ ]:
def check_model_status():
    """Check if models are loaded and ready"""
    try:
        # Check running models/processes
        response = requests.get('http://localhost:11434/api/ps')
        if response.status_code == 200:
            running_models = response.json()
            print("Currently running models:")
            print(json.dumps(running_models, indent=2))
        
        # Test both models with simple prompts
        models_to_test = ["gemma3:4b", "gemma3:1b", "qwen3:4b"]
        
        for model in models_to_test:
            print(f"\nTesting {model}...")
            try:
                response = requests.post('http://localhost:11434/api/generate', 
                                       json={
                                           "model": model,
                                           "prompt": "Hello",
                                           "stream": False,
                                           "options": {"num_predict": 10}
                                       }, timeout=30)
                
                if response.status_code == 200:
                    result = response.json()
                    print(f"✅ {model} responded: {result.get('response', '')}...")
                    # print(f"✅ {model} responded: {result.get('response', '')[:50]}...")
                else:
                    print(f"❌ {model} failed with status: {response.status_code}")
                    print(f"Error: {response.text}")
                    
            except Exception as e:
                print(f"❌ {model} error: {e}")
                
    except Exception as e:
        print(f"Error checking model status: {e}")

check_model_status()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434
DEBUG:urllib3.connectionpool:http://localhost:11434 "GET /api/ps HTTP/1.1" 200 346
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434


Currently running models:
{
  "models": [
    {
      "name": "qwen3:4b",
      "model": "qwen3:4b",
      "size": 4728619520,
      "digest": "2bfd38a7daaf4b1037efe517ccb73d1a3bbd4822cf89f1a82be1569050a114e0",
      "details": {
        "parent_model": "",
        "format": "gguf",
        "family": "qwen3",
        "families": [
          "qwen3"
        ],
        "parameter_size": "4.0B",
        "quantization_level": "Q4_K_M"
      },
      "expires_at": "2025-06-01T15:00:46.7882175+08:00",
      "size_vram": 0
    }
  ]
}

Testing gemma3:4b...


DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/1.1" 200 406
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434


✅ gemma3:4b responded: Hello there! How can I help you today?...

Testing gemma3:1b...


DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/1.1" 200 409
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:11434


✅ gemma3:1b responded: Hello there! How’s your day going so...

Testing qwen3:4b...


DEBUG:urllib3.connectionpool:http://localhost:11434 "POST /api/generate HTTP/1.1" 200 425


✅ qwen3:4b responded: <think>
Okay, the user just said "Hello...




## 4. **Memory and Performance Issues**

Your system might struggle with `qwen3:4b`:



In [ ]:
def check_system_resources():
    """Check system resources and Ollama memory usage"""
    try:
        import psutil
        
        # Check memory usage
        memory = psutil.virtual_memory()
        print(f"Total RAM: {memory.total / (1024**3):.1f} GB")
        print(f"Available RAM: {memory.available / (1024**3):.1f} GB")
        print(f"Used RAM: {memory.used / (1024**3):.1f} GB ({memory.percent}%)")
        
        # Check for Ollama processes
        for proc in psutil.process_iter(['pid', 'name', 'memory_info']):
            if 'ollama' in proc.info['name'].lower():
                memory_mb = proc.info['memory_info'].rss / (1024 * 1024)
                print(f"Ollama process: PID {proc.info['pid']}, Memory: {memory_mb:.1f} MB")
                
    except ImportError:
        print("Install psutil for detailed system monitoring: pip install psutil")
    except Exception as e:
        print(f"Error checking resources: {e}")

check_system_resources()

Total RAM: 15.8 GB
Available RAM: 2.4 GB
Used RAM: 13.4 GB (84.7%)
Ollama process: PID 22492, Memory: 3734.8 MB
Ollama process: PID 28652, Memory: 129.1 MB
Ollama process: PID 37244, Memory: 14.4 MB




## 5. **Fix: Update Flask API with Better Error Handling**

Update your Flask API to handle these model-specific issues:



In [ ]:
@app.route('/api/generate', methods=['POST'])
@require_api_key
def api_generate():
    """Generate text using Ollama with model-specific handling"""
    data = request.get_json()
    
    if not data or 'prompt' not in data:
        return jsonify({'error': 'Prompt is required'}), 400
    
    prompt = data['prompt']
    model = data.get('model', ollama_default_model)
    stream = data.get('stream', False)
    
    # Model-specific timeout and options
    if "qwen" in model.lower():
        timeout = 180
        options = {"num_predict": 100, "temperature": 0.7}
    else:
        timeout = 60
        options = {"num_predict": 150, "temperature": 0.7}
    
    try:
        # Direct Ollama API call with custom timeout
        response = requests.post(f'http://localhost:11434/api/generate', 
                               json={
                                   "model": model,
                                   "prompt": prompt,
                                   "stream": stream,
                                   "options": options
                               }, timeout=timeout)
        
        if response.status_code == 200:
            return jsonify(response.json())
        else:
            return jsonify({
                'error': f'Ollama API error: {response.status_code}',
                'details': response.text
            }), 500
            
    except requests.exceptions.Timeout:
        return jsonify({
            'error': f'Request timeout after {timeout} seconds with model {model}',
            'suggestion': 'Try using a smaller model or reduce the prompt length'
        }), 504
    except Exception as e:
        return jsonify({
            'error': f'Failed to generate response: {str(e)}',
            'model': model
        }), 500

AssertionError: The setup method 'route' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.



## 6. **Recommended Solutions**

1. **Use a smaller Qwen model**:
   ```python
   ollama_manager.pull_gemma_model("qwen2:1.5b")  # Smaller, faster
   ```

2. **Increase timeouts in your Oxygen Positron configuration**

3. **Pre-load the model** by running a simple query first:
   ```python
   # Warm up the model
   ollama_manager.generate_response("Hi", model="qwen3:4b")
   ```

4. **Monitor resource usage** and consider using `gemma3:1b` or `qwen2:1.5b` for better performance on your system.

The issue is likely that `qwen3:4b` requires more computational resources and time to respond, causing timeouts in your Flask API and Oxygen Positron integration.